In [ ]:
# https://www.geeksforgeeks.org/ml-unsupervised-face-clustering-pipeline/
# https://github.com/cppxaxa/FaceRecognitionPipeline_GeeksForGeeks/blob/master/FaceClusteringLibrary.py

# '''
# The ResizeUtils provides resizing function
#            to keep the aspect ratio intact
# Credits: AndyP at StackOverflow'''
# class ResizeUtils:
#     # Given a target height, adjust the image
#     # by calculating the width and resize
#     def rescale_by_height(self, image, target_height,
#                         method = cv2.INTER_LANCZOS4):
         
#         # Rescale `image` to `target_height`
#         # (preserving aspect ratio)
#         w = int(round(target_height * image.shape[1] / image.shape[0]))
#         return (cv2.resize(image, (w, target_height),
#                              interpolation = method))
 
#     # Given a target width, adjust the image
#     # by calculating the height and resize
#     def rescale_by_width(self, image, target_width,
#                         method = cv2.INTER_LANCZOS4):
                             
#         # Rescale `image` to `target_width`
#         # (preserving aspect ratio)
#         h = int(round(target_width * image.shape[0] / image.shape[1]))
#         return (cv2.resize(image, (target_width, h),
#                             interpolation = method))


# resizeUtils = ResizeUtils()

# def AutoResize(frame):
#     resizeUtils = ResizeUtils()

#     height, width, _ = frame.shape

#     if height > 500:
#         frame = resizeUtils.rescale_by_height(frame, 500)
#         AutoResize(frame)
    
#     if width > 700:
#         frame = resizeUtils.rescale_by_width(frame, 700)
#         AutoResize(frame)
    
#     return frame

In [ ]:
# face_cascade = cv2.CascadeClassifier('./data/open_cv_data/haarcascades/haarcascade_frontalface_default.xml')
# ADD PROFILE
# # https://towardsdatascience.com/face-detection-in-2-minutes-using-opencv-python-90f89d7c0f81
# # Convert into grayscale
# gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
# faces = face_cascade.detectMultiScale(gray, 1.1, 4)
# for (x, y, w, h) in faces:
#     cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
# # Display the output
# cv2.imshow('img', frame)
# # cv2.waitKey()
# print(frame.shape)
# frame = AutoResize(frame)
# print(frame.shape)

In [1]:
import cv2
import numpy as np
import pandas as pd
import face_recognition
from pathlib import Path

from sklearn.cluster import DBSCAN


In [2]:
# get all images
p = Path('./data/Sarkob/').glob('**/*.jpg')
files = [x for x in p if x.is_file()]

# TODO: 
* video frame extraction
* resize (?)
* visualization
* 


In [22]:
# https://github.com/cppxaxa/FaceRecognitionPipeline_GeeksForGeeks/blob/02bdc2d7dc8d9cf85f5b932e907723cbe1843fdf/FaceClusteringLibrary.py#L137

df = pd.DataFrame()
for imagePath in files:
    frame  = cv2.imread(str(imagePath))

    # extracting face locations
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    boxes = face_recognition.face_locations(rgb, model="cnn")

    # get face encodings
    encodings = face_recognition.face_encodings(rgb, boxes)

    d = [{"imagePath": imagePath, "loc": box, "encoding": enc} 
        for (box, enc) in zip(boxes, encodings)]

    df_single_image = pd.DataFrame(d)

    df = pd.concat([df, df_single_image])

df = df.reset_index(drop=True)

In [24]:
imagePath

PosixPath('data/Sarkob/46.jpg')

In [29]:
frame  = cv2.imread(str(imagePath))

# extracting face locations
rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
# boxes = face_recognition.face_locations(rgb, model="cnn")

In [30]:
cv2.imshow('img', rgb)

In [27]:
for (x, y, w, h) in boxes:
    cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

In [31]:
# Display the output
cv2.imshow('img', frame)

In [19]:
# cluster the embeddings
clt = DBSCAN(eps=0.1, metric="euclidean", n_jobs=-1)
clt.fit(df['encoding'].values.tolist())


DBSCAN(eps=0.1, n_jobs=-1)

In [20]:
# determine the total number of unique faces found in the dataset
labelIDs = np.unique(clt.labels_)
numUniqueFaces = len(np.where(labelIDs > -1)[0])
print("[INFO] # unique faces: {}".format(numUniqueFaces))


[INFO] # unique faces: 0


In [21]:
clt.labels_

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [10]:
df['labels'] = clt.labels_

In [12]:
df[df['labels'] != -1]

,imagePath,loc,encoding,labels
0,data/Sarkob/63.jpg,"(93, 740, 133, 701)","[-0.014609582722187042, 0.04062148556113243, 0...",0
6,data/Sarkob/63.jpg,"(33, 744, 73, 705)","[-0.10015466064214706, 0.06958091259002686, 0....",0
8,data/Sarkob/88.jpg,"(174, 561, 242, 493)","[-0.10004287213087082, 0.11889319866895676, 0....",0
9,data/Sarkob/88.jpg,"(69, 135, 151, 53)","[-0.1274067610502243, 0.07772970199584961, 0.0...",0
13,data/Sarkob/89.jpg,"(688, 575, 756, 507)","[-0.12351204454898834, 0.0645938366651535, 0.0...",0
19,data/Sarkob/74.jpg,"(143, 423, 211, 355)","[-0.12526649236679077, 0.04904673248529434, 0....",0
22,data/Sarkob/61.jpg,"(281, 223, 349, 154)","[-0.11621767282485962, 0.07237664610147476, 0....",0
23,data/Sarkob/75.jpg,"(136, 234, 218, 152)","[-0.06694430857896805, 0.05297788977622986, 0....",0
24,data/Sarkob/75.jpg,"(254, 82, 311, 25)","[-0.16853153705596924, 0.041529446840286255, 0...",0
26,data/Sarkob/71.jpg,"(148, 154, 216, 85)","[-0.10773828625679016, 0.029197975993156433, 0...",0
